<a href="https://colab.research.google.com/github/EgonFerri/codemotion-challenge/blob/main/codemotion_challenge_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import polars as pl

# Challenge: PokéBattle Champion – Type Multipliers Edition Background


You’re a Codemotion trainer with a team of 6 Pokémon (all from the first two generations), and you’re about to face a single opponent (a real Gen 1 or Gen 2 Pokémon).
Your job is to choose the best Pokémon to battle the opponent by computing a score for each team member based on its stats and type interactions.

# The Challenge

Implement a function

```python
def choose_best_pokemon(team, opponent):
    # your code here
```

where:
- **team** is a dictionary mapping Pokémon names to levels. For example:
  ```python
  team = {
      "pikachu": 32,
      "charmander": 30,
      "bulbasaur": 33,
      "squirtle": 31,
      "eevee": 35,
      "pidgey": 28
  }
  ```
- **opponent** is a dictionary with the keys `"name"` and `"level"`. For example:
  ```python
  opponent = {"name": "geodude", "level": 20}
  ```

Your function must do the following:

1. **Load the Data:**
   - Read **pokemon.csv** to look up each Pokémon’s record by matching its name (case‑insensitively). For every Pokémon you use from the team as well as the opponent, retrieve its `"Attack"`, `"Defense"`, `"Total"`, `"Type 1"`, and `"Type 2"` (if available).  
     (If a Pokémon has no secondary type, its `"Type 2"` field will be an empty string.)
   - Read **attacking_types_chart.csv** to look up for attacking multipliers.

2. **Compute the Score:**
   For each Pokémon in the team, using its stats from **pokemon.csv** and its level from the team dictionary, compute its score using the formula:
   ```python
   score = (Attack * attacking_bonus - opponent_Defense)  + \
           (Defense - opponent_Attack * attacking_bonus)  + \
           5 * (team_level - opponent_level)
   ```

3. **Return the Best Choice:**
   Return the name (the key from the team dictionary) of the Pokémon with the highest computed score. If the team is empty, return `None`.

---

## Additional Notes

- **CSV Loading:**  
  Your solution must read the provided CSV files. You may use Python’s built-in `csv` module or a library like `pandas` to load the files.  
  Make sure to normalize Pokémon names and type names (e.g. convert to lowercase) for successful lookups.

- **Computing Multipliers:**  
  To compute the **attacking_bonus** for a matchup, iterate over every combination of one of your Pokémon’s types (from `"Type 1"` and, if nonempty, `"Type 2"`) and one of the opponent’s types (similarly, `"Type 1"` and `"Type 2"`). Multiply the corresponding values from the `attacking_multiplier` dictionary.


Good luck—and may your chosen Pokémon always have the type advantage!

In [ ]:
def calculate_score(att1, def1, att2, def2, lev1, lev2, attbonus1, attbonus2):
  a = (att1*attbonus1)-def2
  b = def1 - (att2*attbonus2)
  c = 5*(lev1 - lev2)

  return a+b+c

In [ ]:
pokemons = pd.read_csv('pokemon.csv', index_col=0)
types_table = pd.read_csv('attacking_types_chart.csv', index_col=0)
def calculate_attacking_bonus(types_table,types1=(), types2=()):
    m = 1
    for tp in types1:
      for tp2 in types2:
        m = m*types_table[tp2][tp]
    return m

calculate_attacking_bonus(types_table, ['Grass'], ['Fire'])

np.float64(0.5)

In [ ]:
def get_pokemon_data(pokemon_data, pokemon_name):
  pkmn = pokemon_name.capitalize()
  row = pokemons[pokemons['Name']==pkmn].reset_index(drop=True).to_dict()
  types = [row['Type 1'][0]]
  if type(row['Type 2'][0]) == str:
    types.append(row['Type 2'][0])
  return types, row['Attack'][0], row['Defense'][0]
get_pokemon_data(pokemons, 'charmander')

(['Fire'], 52, 43)

In [ ]:
def choose_best_pokemon(team, opponent):
  opp_level = opponent['level']
  opp_name = opponent['name']
  opp_tipi, opp_att, opp_defe = get_pokemon_data(pokemons, opp_name)
  best_score = -1000000
  for pokemon, level in team.items():
    tipi, att, defe = get_pokemon_data(pokemons, pokemon)
    bonus1 = calculate_attacking_bonus(types_table, tipi, opp_tipi)
    bonus2 = calculate_attacking_bonus(types_table, opp_tipi, tipi)
    score = calculate_score(att, defe, opp_att, opp_defe, level, opp_level, bonus1, bonus2)
    if score>best_score:
      best_score = score
      best_pokemon = pokemon
  print(f'for {opp_name} the best choice is {best_pokemon}')
  return best_pokemon

In [ ]:
team = {
        "pikachu": 32,
        "charmander": 30,
        "bulbasaur": 33,
        "squirtle": 31,
        "eevee": 35,
        "pidgey": 28
    }
opponent = {"name": "geodude", "level": 20}

best_choice = choose_best_pokemon(team, opponent)
print(best_choice)

for geodude the best choice is squirtle
squirtle


# TEST

In [ ]:

def test1():
    # Team: Gen 1 Pokémon; Opponent: Geodude (Rock/Ground)
    team = {
        "pikachu": 32,
        "charmander": 30,
        "bulbasaur": 33,
        "squirtle": 31,
        "eevee": 35,
        "pidgey": 28
    }
    opponent = {"name": "geodude", "level": 20}
    # Based on the formula and multipliers, assume the best score is for "squirtle"
    assert choose_best_pokemon(team, opponent) == "squirtle"

def test2():
    # Team: Gen 1 Pokémon; Opponent: Onix (Rock/Ground)
    team = {
        "pikachu": 32,
        "charmander": 30,
        "bulbasaur": 33,
        "squirtle": 31,
        "eevee": 35,
        "pidgey": 28
    }
    opponent = {"name": "onix", "level": 22}
    result = choose_best_pokemon(team, opponent)
    assert result in team

def test3():
    # Team: Gen 2 Pokémon; Opponent: Umbreon (Dark)
    team = {
        "chikorita": 30,
        "cyndaquil": 30,
        "totodile": 30,
        "pichu": 25,
        "togepi": 26,
        "mareep": 27
    }
    opponent = {"name": "umbreon", "level": 30}
    result = choose_best_pokemon(team, opponent)
    assert result in team

def test4():
    # Team: Mixed Gen; Opponent: Murkrow (Dark/Flying)
    team = {
        "pikachu": 32,
        "bulbasaur": 33,
        "squirtle": 31,
        "chikorita": 30,
        "cyndaquil": 30,
        "totodile": 30
    }
    opponent = {"name": "murkrow", "level": 25}
    result = choose_best_pokemon(team, opponent)
    assert result in team

def test5():
    # Team: Gen 1 Pokémon; Opponent: Gyarados (Water/Flying)
    team = {
        "machop": 29,
        "gastly": 30,
        "onix": 27,
        "hitmonlee": 31,
        "jigglypuff": 25,
        "snorlax": 35
    }
    opponent = {"name": "gyarados", "level": 35}
    result = choose_best_pokemon(team, opponent)
    assert result in team

def test6():
    # Team: Early Gen 1 Pokémon; Opponent: Alakazam (Psychic)
    team = {
        "pidgey": 20,
        "rattata": 20,
        "ekans": 22,
        "sandshrew": 23,
        "sandshrew": 100,
        "clefairy": 24
    }
    opponent = {"name": "alakazam", "level": 28}
    result = choose_best_pokemon(team, opponent)
    assert result in team

def test7():
    # Team: Gen 2 Pokémon; Opponent: Feraligatr (Water)
    team = {
        "chikorita": 30,
        "cyndaquil": 30,
        "totodile": 30,
        "bellossom": 34,
        "hoothoot": 22,
        "ledyba": 20
    }
    opponent = {"name": "feraligatr", "level": 32}
    result = choose_best_pokemon(team, opponent)
    assert result in team

def test8():
    # Team: Gen 1 Pokémon; Opponent: Dragonite (Dragon/Flying)
    team = {
        "pikachu": 32,
        "charmander": 30,
        "bulbasaur": 33,
        "squirtle": 31,
        "jigglypuff": 25,
        "meowth": 28
    }
    opponent = {"name": "dragonite", "level": 40}
    result = choose_best_pokemon(team, opponent)
    assert result in team

# Run all tests
if __name__ == "__main__":
    tests = [test1, test2, test3, test4, test5, test6, test7, test8]
    for i, test in enumerate(tests, 1):
        try:
            test()
            print(f"Test {i} passed.")
        except AssertionError:
            print(f"Test {i} failed.")


for geodude the best choice is squirtle
Test 1 passed.
for onix the best choice is squirtle
Test 2 passed.
for umbreon the best choice is totodile
Test 3 passed.
for murkrow the best choice is pikachu
Test 4 passed.
for gyarados the best choice is snorlax
Test 5 passed.
for alakazam the best choice is sandshrew
Test 6 passed.
for feraligatr the best choice is bellossom
Test 7 passed.
for dragonite the best choice is jigglypuff
Test 8 passed.
